In [1]:
import csv
import subprocess
import getpass
import os
import gzip
from os import listdir
from os.path import isfile, join
from SciServer import Authentication

myUserName = Authentication.getKeystoneUserWithToken(Authentication.getToken()).userName
passwd = getpass.getpass('Password for ' + myUserName + ': ')
userstring = "username=" + myUserName + ",workgroup=win,uid=idies,password=" + passwd
projectname = input("Please enter the name of your SAFE Desktop project folder : ") 
dir = "//cloud.nas.jh.edu/sddesktop$/" + projectname
devnull = open(os.devnull, 'w')
subprocess.run(["sudo", "mkdir", "/home/idies/workspace/SAFE"], capture_output=False)
subprocess.run(["sudo", "chown", "idies:idies", "/home/idies/workspace/SAFE/"], capture_output=False)
try:
    subprocess.run(["sudo", "mount", "-t", "cifs", dir, "/home/idies/workspace/SAFE/", "-o", userstring], stdout=devnull, stderr=devnull)
except FileNotFoundError as e:
    print(e)

Password for mkim239:  ········
Please enter the name of your SAFE Desktop project folder :  LCICM


In [39]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
import gc
import pyarrow
import fastparquet

def high_pass_filter(signal, cutoff=0.5, fs=500, order=3):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype="high", analog=False)
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

def process_ecg(input_path, output_path):
    df = pd.read_parquet(input_path)

    for column in df.columns:
        df[column] = df[column].fillna(df[column].mean())
        df[column] = high_pass_filter(df[column].values)

    # impute missing values
    df.to_parquet(output_path)

In [41]:
ids = pd.read_csv("SAFE/hf_ecg_list_id.csv")["formatted_time"]
print(len(ids))
print(ids[0])
extracted = []

21439
13:45:00_11_01_2110_18106347


In [44]:
# extract advanced ecg features for pos HF patients 
# extract in chunks (# = counter) to preserve kernel
counter = 0
batch = 1000 # change this
for id in ids:
    if id not in extracted:
        path = "SAFE/Databases/mimic-iv-ecg-parquet/" + id.split("_")[-1] + "/" + id + ".parquet"
        if os.path.exists(path):    
            # do preprocessing
            process_ecg(path, "SAFE/Databases/ecg_preprocessed/" + id)

            # add counters
            counter+=1
            extracted.append(id)

gc.collect()

0

In [45]:
print(len(os.listdir("SAFE/Databases/ecg_preprocessed")))

21425
